# Projet 8 : Images preprocessing spark

### Uploading Database on S3

In [ ]:
#!pip install --upgrade --user awscli   

In [ ]:
# source = "/mnt/d/Openclassrooms/Projets/P8_KOKORA_AMON/archive/fruits-360-original-size/fruits-360-original-size/Training/"
# destination  = "s3://jpkokora/ocr_p08/Training"

In [ ]:
# !aws s3 sync source destination

### Libraries and data loading

In [ ]:
# !pip install pyspark

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.ml.linalg import Vectors
# from pyspark.sql.functions import *
# import matplotlib.pyplot as plt
# import numpy as np

In [ ]:
# from joblibspark import register_spark
# from sklearn.utils import parallel_backend

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Image preprcocessing") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
spark

In [4]:
# Data 
path="/mnt/d/Openclassrooms/Projets/P8_KOKORA_AMON/archive/fruits-360-original-size/fruits-360-original-size/Training/apple_6/"
#path = "s3://jpkokora/ocr_p8/Training/**"
# images = spark.read.format("image").load(path)
# mysplit = udf(lambda z: z.split('/')[-2]) 
# images = images.withColumn('label',mysplit(col('image.origin')))

In [6]:
#image.printSchema()

In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, smart_resize
from tensorflow.keras.models import load_model

base_model_mn2 = MobileNetV2(weights="imagenet", input_shape=(224, 224, 3))
# remove the output layer
my_mn2 = Model(inputs=base_model_mn2.inputs, outputs=base_model_mn2.layers[-2].output)

my_mn2.save('/mnt/d/model_mobile_v2')

INFO:tensorflow:Assets written to: /mnt/d/model_mobile_v2/assets


In [12]:
def transfer_learning(iterator):
    from tensorflow.keras.models import 
    # model = load_model("s3://jpkokora/ocr_p8/model_mobilenet2.h5")
    model = load_model("/mnt/d/model_mobile_v2")
    for record in iterator:
        yield model.predict(record)[0].tolist()

In [16]:
def features_extractor(data, spark_obj):
    from pyspark.sql.functions import monotonically_increasing_id, row_number
    from pyspark.sql.window import Window
    from pyspark.sql.functions import udf, col
    from pyspark.ml.image import ImageSchema
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
    from tensorflow.keras.preprocessing.image import load_img, img_to_array, smart_resize
    
    images = spark_obj.read.format("image").load(path)
    mysplit = udf(lambda z: z.split('/')[-2]) 
    images = images.withColumn('label',mysplit(col('image.origin')))
    labels = images.select('label')
    data_rdd = images.rdd
    #libraries
    
    ImageSchema.toNDArray(data_rdd.first()['image'])

    data_rdd = data_rdd.map(lambda x : ImageSchema.toNDArray(x['image'])) \
        .map(lambda x : smart_resize(x, size=(224,224))) \
        .map(lambda x: x.reshape((1, x.shape[0], x.shape[1], x.shape[2]))) \
        .map(lambda x: preprocess_input(x)) \
        .mapPartitions(transfer_learning) 
    
    data_rdd = spark_obj.createDataFrame(data_rdd)
    # since there is no common column between these two dataframes add row_index so that it can be joined
    data_rdd=data_rdd.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
    labels=labels.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))

    data_rdd = labels.join(data_rdd, on=["row_index"]).drop("row_index")

    return data_rdd

In [19]:
data = features_extractor(path, spark).persist()

In [20]:
data.count()

315

In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.feature import PCA

assembler = VectorAssembler(
  inputCols=data.columns[1:], outputCol="pfeatures", 
)
scaler = StandardScaler(inputCol="pfeatures", outputCol="scaledFeatures",
                         withStd=True, withMean=True)
pca = PCA(k=850, inputCol="scaledFeatures", outputCol="pcaFeatures")


In [22]:
pipeline = Pipeline(stages=[assembler,scaler,pca]).fit(data)

In [23]:
data = pipeline.transform(data).select('label','pcaFeatures')

In [24]:
type(data), data.printSchema(), 

root
 |-- label: string (nullable = true)
 |-- pcaFeatures: vector (nullable = true)



(pyspark.sql.dataframe.DataFrame, None)

In [25]:
def extract(row):
    return (row.label, ) + tuple(row.pcaFeatures.toArray().tolist())

data = data.rdd.map(extract).toDF(["label"])

In [38]:
#saving data
# import os
data.write.option("header","true").csv("/mnt/d/csv/data_cleaned",mode="overwrite")
# data.write.option("header","true").csv("s3://jpkokora/ocr_p8/data_preproceed",mode="overwrite")

In [35]:
re = spark.read.format('csv').options(header='true', inferSchema='true').load("/mnt/d/csv/data_cleaned.csv")

AnalysisException: Path does not exist: file:/mnt/d/csv/data_cleaned.csv